In [2]:
from transformers import pipeline

# Charger le modèle DistilBERT entraîné
model_path = r"C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/models/distilbert"
token_classifier = pipeline("token-classification", model=model_path, aggregation_strategy="simple")

# Liste de phrases à tester
test_sentences = [
    "Je pars de Paris et j'arrive à Marseille.",
    "Je vais à Bordeaux en partant de Toulouse.",
    "Mon trajet va de VILLIERS SUR LOIR à JARNY.",
    "Nous partons de Lyon et nous arrivons à Nice.",
    "Le départ est prévu à Lille, et l'arrivée se fera à Strasbourg.",
    "Mon itinéraire inclut un départ de Montpellier et une arrivée à Perpignan.",
    "Le train partira de Nantes pour arriver à Rennes.",
    "Un vol de Genève à Zurich est prévu demain matin.",
    "Je vais de Grenoble à Dijon en voiture.",
    "Je voyage de Bruxelles à Amsterdam ce week-end.",
    "L'itinéraire commence à Marseille et se termine à Toulon.",
    "Nous prenons un train de Rouen pour aller au Havre.",
    "Le départ sera à Limoges et l'arrivée à Poitiers.",
    "Je quitte Monaco pour aller à Menton.",
    "Le bus va de Caen à Cherbourg.",
    "Il part de Bayonne et arrive à Biarritz.",
    "Je vais de Chamonix à Annecy.",
    "Nous roulons de Bastia à Ajaccio.",
    "Le ferry part de Calais pour arriver à Douvres.",
    "Je pars de Bordeaux et j'arrive à Paris.",
    "Notre vol va de Madrid à Barcelone.",
    "Un trajet en voiture de Milan à Rome est prévu.",
    "Le départ est de Zurich et l'arrivée est à Lausanne.",
    "Je vais à Londres depuis Manchester.",
    "Le train va de Tokyo à Kyoto.",
    "Je vais de Nantes à Lyon en train.",
    "Mon avion part de Nice pour aller à Paris.",
    "Je prends un bus de Toulouse à Bordeaux.",
    "Nous partons en voiture de Lyon à Marseille.",
    "Je pars de Rennes pour arriver à Brest.",
    "Le train part de Grenoble et arrive à Annecy.",
    "Je vais de Lille à Nantes en avion.",
    "Le trajet est de Strasbourg à Lyon.",
    "Le bus va de Dijon à Auxerre.",
    "Je vais à Nice depuis Paris.",
    "Le départ se fait de Tours et l'arrivée est à Nantes.",
    "Je voyage de Lille à Metz en train.",
    "Je vais de Clermont-Ferrand à Lyon en voiture.",
    "Le départ est prévu de Grenoble pour aller à Lyon.",
    "Un trajet en train de Paris à Lille est prévu.",
    "Je pars de Toulouse et je vais à Perpignan.",
    "L'itinéraire commence à Aix-en-Provence et se termine à Marseille.",
    "Le vol de Nice à Zurich est prévu pour demain.",
    "Je quitte Lyon pour aller à Toulouse.",
    "Le trajet en train va de Rennes à Nantes.",
    "Je vais de Marseille à Bordeaux en train.",
    "Mon départ est de Dijon et j'arrive à Lyon.",
    "Le train va de Lille à Calais.",
    "Je pars de Montpellier et je vais à Perpignan.",
    "Le bus part de Paris pour aller à Rouen.",
    "Nous roulons de Strasbourg à Lyon.",
    "Je vais de Nantes à Bordeaux en voiture.",
    "Je pars de Toulon pour aller à Marseille.",
    "Mon itinéraire commence à Paris et se termine à Lille.",
    "Le départ est de Marseille et l'arrivée est à Lyon.",
    "Je vais de Tours à Poitiers en train.",
    "Un trajet en voiture va de Lille à Paris.",
    "Je voyage de Lyon à Dijon en train.",
    "Je pars de Toulouse et j'arrive à Bordeaux.",
    "Nous partons de Nice et nous allons à Marseille.",
    "Le départ est de Paris et l'arrivée est à Lille.",
    "Je pars de Lyon et j'arrive à Nice.",
    "Je vais de Marseille à Toulon en voiture.",
    "Un vol va de Paris à Genève.",
    "Le train part de Bordeaux et arrive à Nantes.",
    "Je vais de Paris à Lyon en voiture.",
    "Le trajet en voiture est de Toulouse à Nice.",
    "Je vais à Nantes depuis Lyon.",
    "Le bus part de Marseille pour arriver à Toulon.",
    "Je vais de Rouen à Paris en train."
]

# Fonction pour évaluer le modèle
def evaluate_distilbert(token_classifier, sentences):
    total_phrases = len(sentences)
    dep_city_count = 0
    arr_city_count = 0

    for sentence in sentences:
        outputs = token_classifier(sentence)
        dep_city = None
        arr_city = None
        print(f"Analyse de la phrase : {sentence}")
        print("Entités trouvées :")
        
        # Extraction des entités
        for ent in outputs:
            entity_label = ent.get('entity_group', ent.get('entity'))  # Récupérer l'étiquette correcte
            word = ent.get('word', '')
            print(f"- {word} ({entity_label})")
            
            if entity_label in ["B-DEP", "I-DEP"]:
                dep_city = word
                dep_city_count += 1
            elif entity_label in ["B-ARR", "I-ARR"]:
                arr_city = word
                arr_city_count += 1

        # Ajustement en fonction de la phrase
        if "départ" in sentence.lower() and arr_city and not dep_city:
            dep_city = arr_city
            arr_city = None
        
        if "arrivée" in sentence.lower() and dep_city and not arr_city:
            arr_city = dep_city
            dep_city = None

    # Calcul des pourcentages
    dep_city_percentage = (dep_city_count / total_phrases) * 100
    arr_city_percentage = (arr_city_count / total_phrases) * 100

    return total_phrases, dep_city_count, arr_city_count, dep_city_percentage, arr_city_percentage

# Obtenir les résultats
total_phrases, dep_city_count, arr_city_count, dep_city_percentage, arr_city_percentage = evaluate_distilbert(token_classifier, test_sentences)

# Résumé des résultats
print("\nRésumé des résultats :")
print(f"Nombre total de phrases analysées : {total_phrases}")
print(f"Nombre de villes de départ identifiées : {dep_city_count} ({dep_city_percentage:.2f}%)")
print(f"Nombre de villes d'arrivée identifiées : {arr_city_count} ({arr_city_percentage:.2f}%)")

Analyse de la phrase : Je pars de Paris et j'arrive à Marseille.
Entités trouvées :
- paris (DEP)
- marseille (ARR)
Analyse de la phrase : Je vais à Bordeaux en partant de Toulouse.
Entités trouvées :
- bordeaux (ARR)
- toulouse (DEP)
Analyse de la phrase : Mon trajet va de VILLIERS SUR LOIR à JARNY.
Entités trouvées :
- villiers sur loir (DEP)
- jar (ARR)
- ##ny (ARR)
Analyse de la phrase : Nous partons de Lyon et nous arrivons à Nice.
Entités trouvées :
- lyon (DEP)
- ##rivons (DEP)
- nice (ARR)
Analyse de la phrase : Le départ est prévu à Lille, et l'arrivée se fera à Strasbourg.
Entités trouvées :
- lille (ARR)
- , (ARR)
- strasbourg (ARR)
Analyse de la phrase : Mon itinéraire inclut un départ de Montpellier et une arrivée à Perpignan.
Entités trouvées :
- montpellier (DEP)
- per (ARR)
- ##pi (ARR)
- ##gnan (ARR)
Analyse de la phrase : Le train partira de Nantes pour arriver à Rennes.
Entités trouvées :
- nantes (DEP)
- ren (ARR)
- ##nes (ARR)
Analyse de la phrase : Un vol de Genèv